In [1]:
from multiprocessing import Process, Queue
import numpy as np
import time

In [2]:
class DummyAgent():
    def __init__(self, m):
        self.m = m
    
    def start_new_task(self, inqueue, outqueue):
        img = inqueue # First message is the image to segment
        print("Agent {} segmenting the image".format(self.m))
        self.answer_to_predictions(inqueue, outqueue)
    
    def answer_to_predictions(self, inqueue, outqueue):
        print("Agent {} ready to negotiate".format(self.m))
        while True:
            agreement = inqueue.get()
            if agreement is None:
                print("Agent exiting negotiation")
                break
            
            proposal = np.random.random()
            print("I'm {} working on agreement {}. Proposing {}".format(self.m, agreement, proposal))
            outqueue.put(proposal)
        
class DummyMediator():
    def __init__(self, agents):
        # Spawn some agents
        self.agents = agents
        self.last_agreement = 0
        self.output_queues = [Queue() for a in agents]
        self.input_queues = [Queue() for a in agents]
        self.last_proposals = {a: None for a in range(4)}
        self.agentpool = None
        
    def start_new_task(self, img=1234, n=130):
        # Terminating all the pending agent tasks
        if self.agentpool is not None:
            [a.terminate() for a in self.agentpool]
        self.agentpool = [Process(target=a.start_new_task, args=(self.output_queues[i], self.input_queues[i])) for i, a in enumerate(agents)]
        # Starting the new processes
        for p in self.agentpool:
            p.start()
        # Sumbitting the task
        [q.put(img) for q in self.output_queues]
        # Switching to negotiation
        return self.negotiate(n)
            
    def negotiate(self, n):
        for i in range(n):
            print("Negotiation Step {}".format(n))
            # Wait for responses from every agent
            proposals = [q.get() for q in self.input_queues]
            print(proposals)
            # Calculate a new agreement
            agr = np.mean(proposals)
            if i <= n: 
                # Publish an agreement
                [q.put(agr) for q in self.output_queues]
            else:
                return agr
        
    def stop(self):
        for q in self.output_queues:
            q.put(None)



In [3]:
modelnames = list(range(4))
agents = [DummyAgent(m) for m in modelnames]
mediator = DummyMediator(agents)

In [4]:
mediator.start_new_task()

Agent 0 segmenting the image
Agent 1 segmenting the image
Agent 0 ready to negotiate
Agent 1 ready to negotiate
Agent 2 segmenting the image
Agent 2 ready to negotiate
I'm 0 working on agreement 1234. Proposing 0.2158830506106656
Agent 3 segmenting the image
I'm 1 working on agreement 1234. Proposing 0.7377824502876288
Agent 3 ready to negotiate
I'm 2 working on agreement 1234. Proposing 0.49979604636892105
I'm 3 working on agreement 1234. Proposing 0.6434903533816629
I'm 0 working on agreement 0.5242379751622196. Proposing 0.07738652929608725
I'm 1 working on agreement 0.5242379751622196. Proposing 0.23487978008087806
I'm 3 working on agreement 0.5242379751622196. Proposing 0.6519587324298792
I'm 2 working on agreement 0.5242379751622196. Proposing 0.10065194500406072
I'm 1 working on agreement 0.26621924670272634. Proposing 0.05882399875628053
I'm 0 working on agreement 0.26621924670272634. Proposing 0.21574233545066623
I'm 2 working on agreement 0.26621924670272634. Proposing 0.6188

In [ ]:
mediator.stop()